<a href="https://colab.research.google.com/github/gforconi/UTNIA2025/blob/main/NLP_3_langchain_conversaciones_rag_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# LangChain paso a paso: conversación, memoria y RAG

Este notebook muestra una **evolución progresiva** del uso de LangChain en tres etapas:

1. **Conversación simple**: un ejemplo básico con un LLM.
2. **Conversación con memoria (BufferMemory)**: extendemos el ejemplo para mantener historial.
3. **Conversación con RAG**: cargamos documentos, creamos un índice vectorial (Chroma) y consultamos con **conversational RAG**.

> **Requisitos**: Python 3.10+ y una clave de API de OpenAI en la variable de entorno `OPENAI_API_KEY`.


## 0) Instalación y configuración


### (Opcional) Usar modelo local con Ollama en lugar de OpenAI
Si querés correr **local**, activá `USE_OLLAMA = True` y asegurate de tener `ollama` corriendo en `http://localhost:11434`.


In [ ]:

# Selector de proveedor LLM: OpenAI (default) u Ollama (local)
USE_OLLAMA = False  # ← cambiá a True para usar Ollama local

from langchain_openai import ChatOpenAI
try:
    if USE_OLLAMA:
        from langchain_ollama import ChatOllama
        llm_factory = lambda: ChatOllama(model="llama3", temperature=0.1)
    else:
        llm_factory = lambda: ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    print("Proveedor listo:", "Ollama" if USE_OLLAMA else "OpenAI")
except Exception as e:
    raise RuntimeError("Error al configurar el proveedor LLM. ¿Instalaste 'langchain-ollama' si USE_OLLAMA=True?") from e


In [ ]:

# ✳️ Instala dependencias (ejecuta esta celda)
# Sugerencia: usa un entorno virtual (venv/conda) antes de instalar.
!pip install -U langchain langchain-openai langchain-community langchain-text-splitters chromadb pypdf tiktoken
# Opcional: FAISS en lugar de Chroma
# !pip install faiss-cpu


  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached tiktoken-0.11.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.10.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cach

In [ ]:

# 🔐 Configura tu clave de OpenAI
import os

# Opción A (recomendada): exporta la variable en tu sistema antes de abrir el notebook:
#   Linux/Mac: export OPENAI_API_KEY="tu_api_key"
#   Windows (PowerShell): setx OPENAI_API_KEY "tu_api_key"
#
# Opción B: establece la clave directamente aquí (no recomendado para producción):
# os.environ["OPENAI_API_KEY"] = "tu_api_key"

# Opción B
os.environ["OPENAI_API_KEY"] = "tu_api_key_aqui"

if not os.getenv("OPENAI_API_KEY"):
    print("⚠️ No se encontró OPENAI_API_KEY en el entorno. Configúrala antes de continuar.")
else:
    print("✅ OPENAI_API_KEY detectada.")


✅ OPENAI_API_KEY detectada.



## 1) Conversación simple con LangChain

En esta primera sección creamos una mini **pipeline** con:
- `ChatPromptTemplate` → define el prompt.
- `ChatOpenAI` → el modelo de chat.
- `StrOutputParser` → convierte la respuesta a `str` para imprimir fácilmente.

La **LCEL (LangChain Expression Language)** permite conectar componentes con el operador `|`.


In [ ]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Modelo de chat (elige el que prefieras; gpt-4o-mini es rápido y económico)
# model = ChatOpenAI(model="gpt-4o-mini", temperature=1)
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt de sistema + entrada del usuario
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente útil y conciso. Responde en español."),
    ("human", "{input}")
])

# Construimos la cadena: prompt -> modelo -> parser de string
simple_chain = prompt | model | StrOutputParser()

# 👇 Prueba rápida
respuesta = simple_chain.invoke({"input": "Hola, ¿quién eres y qué puedes hacer?"})
print(respuesta)


Hola, soy un asistente virtual diseñado para ayudarte con información y responder a tus preguntas. Puedo ofrecerte datos, consejos, explicaciones sobre diversos temas y más. ¿En qué puedo ayudarte hoy?



**Qué observar**  
- `invoke(...)` ejecuta la cadena de principio a fin.  
- Cambia `temperature` para controlar la creatividad.  
- Modifica el texto de `system` para ajustar el "rol" del asistente.


### Temperature

Con `temperature = 1`

In [ ]:
respuesta = simple_chain.invoke({"input": "Que tal esta la clase de Inteligencia Artificial hoy?"})
print(respuesta)

No tengo información en tiempo real, pero puedo ayudarte con conceptos, temas y preguntas sobre la clase de Inteligencia Artificial. ¿Qué aspecto específico te gustaría conocer?


In [ ]:
respuesta = simple_chain.invoke({"input": "Que tal esta la clase de Inteligencia Artificial hoy?"})
print(respuesta)

No tengo acceso a información en tiempo real, pero generalmente las clases de Inteligencia Artificial se centran en temas como el aprendizaje automático, redes neuronales, procesamiento de lenguaje natural y ética en IA. Si quieres saber sobre un tema específico, dime y con gusto te ayudo.


Con `temperature = 0`

In [ ]:
respuesta = simple_chain.invoke({"input": "Que tal esta la clase de Inteligencia Artificial hoy?"})
print(respuesta)

No tengo acceso a información en tiempo real, pero generalmente las clases de Inteligencia Artificial suelen ser muy interesantes y abarcan temas como aprendizaje automático, redes neuronales y procesamiento de lenguaje natural. ¿Te gustaría saber algo específico sobre el tema?


In [ ]:
respuesta = simple_chain.invoke({"input": "Que tal esta la clase de Inteligencia Artificial hoy?"})
print(respuesta)

No tengo acceso a información en tiempo real, pero generalmente las clases de Inteligencia Artificial suelen ser muy interesantes y abarcan temas como aprendizaje automático, redes neuronales y procesamiento de lenguaje natural. ¿Te gustaría saber algo específico sobre el tema?


`temperature = 0.0`

El modelo es **determinista**: casi siempre devuelve la misma respuesta para la misma entrada.
Útil cuando buscás **precisión** y consistencia (ej. generar SQL, código, respuestas factuales).

**Valores bajos (0.1 – 0.3)**

El modelo varía un poco sus respuestas, pero sigue siendo bastante confiable.
Ideal para **chatbots serios**, resúmenes o asistencia técnica.

**Valores medios (0.5 – 0.7)**

Las respuestas son más variadas, con sinónimos, estilos distintos o explicaciones alternativas.
Útil en contextos de **escritura creativa** o **brainstorming**.

**Valores altos (0.8 – 1.0 o más)**

El modelo se vuelve **muy creativo y arriesgado**, pudiendo inventar cosas o desviarse del tema.
Puede servir para poesía, storytelling, generación de ideas “locas”.
Pero en tareas técnicas, aumenta la probabilidad de errores.


## 2) Conversación con memoria (ConversationBufferMemory)

Ahora extendemos el ejemplo para **recordar el historial** de la charla usando `ConversationBufferMemory` y `ConversationChain`.

> **Nota**: `ConversationBufferMemory` está **deprecado** en LangChain (se mantendrá hasta la versión 1.0). Sigue siendo útil para aprender; para proyectos nuevos, LangChain recomienda usar `RunnableWithMessageHistory` o enfoques basados en **LCEL** (los verás en la sección 3).

**Objetivo**: que el bot entienda referencias como “eso”, “lo anterior”, “ella/él”, etc.


In [ ]:

from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

# Guardamos TODO el historial de mensajes en memoria
memory = ConversationBufferMemory(return_messages=True)

# La ConversationChain une LLM + memoria
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

# 🗣️ Diálogo de ejemplo
print("Usuario: Hola, soy Ana y me gusta Python.")
out1 = conversation.invoke("Hola, soy Ana y me gusta Python.")
print("Asistente:", out1["response"])

print("\nUsuario: ¿Qué lenguaje dije que me gustaba?")
out2 = conversation.invoke("¿Qué lenguaje dije que me gustaba?")
print("Asistente:", out2["response"])

# Puedes inspeccionar el buffer de memoria
print("\n--- Memoria acumulada ---")
for m in memory.chat_memory.messages:
    print(type(m).__name__, "→", m.content)


/var/folders/ck/qx4468vx3sx6bf4nt0smfg2c0000gn/T/ipykernel_86485/261983815.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)
/var/folders/ck/qx4468vx3sx6bf4nt0smfg2c0000gn/T/ipykernel_86485/261983815.py:11: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(llm=llm, memory=memory, verbose=True)


Usuario: Hola, soy Ana y me gusta Python.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hola, soy Ana y me gusta Python.
AI:

> Finished chain.
Asistente: ¡Hola, Ana! Encantado de conocerte. Python es un lenguaje de programación muy versátil y popular. ¿Qué es lo que más te gusta de Python? ¿Estás trabajando en algún proyecto en particular o aprendiendo algo nuevo?

Usuario: ¿Qué lenguaje dije que me gustaba?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.



**Qué observar**  
- La memoria **no** es persistente entre ejecuciones del notebook; vive en RAM.  
- Para memorias largas o persistentes, considera `ConversationSummaryMemory` o guardar/recargar historiales.  
- En la sección 3 usamos la alternativa moderna con `RunnableWithMessageHistory`.


`ConversationBufferMemory` está deprecado en LangChain (se mantiene hasta langchain==1.0.0), así que lo muestro para aprender pero en la sección 3 uso el enfoque moderno con RunnableWithMessageHistory y history-aware retriever.


## 3) Conversación con RAG (consultas a una base vectorial)

En esta sección implementamos un **RAG conversacional**:
1. Cargamos documentos locales (carpeta `./data`).
2. Los partimos en fragmentos.
3. Creamos embeddings y un **índice vectorial** con **Chroma**.
4. Usamos un **retriever** que busca los pasajes más relevantes.
5. Armamos una cadena **history-aware** (consciente del historial) para contextualizar preguntas de seguimiento.
6. Mantenemos el historial con `RunnableWithMessageHistory`.

> Si no tienes documentos, crearemos **automáticamente** algunos de ejemplo en `./data`.


In [ ]:

# 3.1) Carga de documentos
from pathlib import Path
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

# Si la carpeta está vacía, creamos documentos de ejemplo
if not any(data_dir.iterdir()):
    (data_dir / "intro_rag.txt").write_text(
        "RAG (Retrieval-Augmented Generation) combina recuperación de contexto con generación. "
        "Permite responder con información actualizada sin reentrenar el modelo."
    )
    (data_dir / "langchain_nociones.md").write_text(
        "# Nociones de LangChain\n"
        "- LCEL permite encadenar componentes con el operador |.\n"
        "- Los retrievers devuelven documentos relevantes.\n"
        "- Los vector stores como Chroma y FAISS almacenan embeddings."
    )

# Cargamos .txt y .md con TextLoader, y .pdf con PyPDFLoader
loaders = []
# Text/Markdown
loaders.append(DirectoryLoader(str(data_dir), glob="**/*.txt", loader_cls=TextLoader, show_progress=True))
loaders.append(DirectoryLoader(str(data_dir), glob="**/*.md", loader_cls=TextLoader, show_progress=True))
# PDF
for pdf in data_dir.glob("**/*.pdf"):
    loaders.append(PyPDFLoader(str(pdf)))

docs = []
for loader in loaders:
    docs.extend(loader.load())

print(f"Documentos cargados: {len(docs)}")


100%|██████████| 1/1 [00:00<00:00, 4691.62it/s]

Documentos cargados: 2


In [ ]:

# 3.2) Particionado de documentos en chunks
# En versiones recientes, los 'splitters' residen en 'langchain_text_splitters'
try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    # Fallback si usas una versión anterior
    from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800, # Cada fragmento tendrá como máximo 800 caracteres.
    chunk_overlap=120, # Cada fragmento se solapa con el siguiente en 120 caracteres.
    add_start_index=True # Añade el índice de inicio del chunk en el documento original (útil para referencias).
)
splits = text_splitter.split_documents(docs)
print(f"Chunks creados: {len(splits)}")


Chunks creados: 2


`chunk_size=800`

Cada fragmento tendrá como máximo 800 caracteres.
Esto ayuda a que el LLM no reciba textos demasiado largos en una sola pasada.

`chunk_overlap=120`

Cada fragmento se solapa con el siguiente en 120 caracteres.
Sirve para que no se “corte” información importante justo en el límite entre dos chunks.
Ejemplo: si un párrafo empieza al final de un chunk, el solapamiento asegura que también aparezca al inicio del siguiente.

`add_start_index=True`

Agrega un índice de inicio (posición dentro del texto original) en la metadata de cada fragmento.
Eso es útil si después querés rastrear en qué parte exacta del documento estaba el chunk.

In [ ]:

# 3.3) Embeddings + Vector Store (Chroma)
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embedding = OpenAIEmbeddings(model="text-embedding-3-small")
persist_dir = "chroma_db"

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_dir  # para que puedas reusar el índice
)

retriever = vectordb.as_retriever(search_kwargs={"k": 4})
print("Vector store listo ✓")


Vector store listo ✓


In [ ]:

# 3.4) Cadena de RAG con historial (enfoque recomendado)
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

llm = llm_factory()

# Prompt que reescribe la consulta considerando el historial (si lo hay)
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", "Reescribe la consulta del usuario para búsqueda, teniendo en cuenta el historial."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

history_aware_retriever = create_history_aware_retriever(
    llm=llm,
    retriever=retriever,
    prompt=contextualize_q_prompt
)

# Prompt final para responder con los documentos recuperados
qa_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Eres un asistente experto. Usa EXCLUSIVAMENTE el 'contexto' proporcionado para responder en español. "
     "Si la respuesta no está en el contexto, di claramente que no aparece.\n\n"
     "===== CONTEXTO =====\n{context}\n=====================\n"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])


combine_docs_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, combine_docs_chain)

print("Cadenas de RAG construidas ✓")


Cadenas de RAG construidas ✓


In [ ]:

# 3.5) Añadir memoria de conversación con RunnableWithMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory  # almacenamiento simple en memoria

# Diccionario {session_id: ChatMessageHistory}
_session_store = {}

def get_session_history(session_id: str):
    if session_id not in _session_store:
        _session_store[session_id] = ChatMessageHistory()
    return _session_store[session_id]

rag_with_history = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",  # la clave por defecto para create_retrieval_chain
)

# Demostración de diálogo multi-turno con el mismo session_id
config = {"configurable": {"session_id": "demo"}}

def conversar(pregunta: str):
    resp = rag_with_history.invoke({"input": pregunta}, config=config)
    print("→", resp["answer"])
    # Si quieres inspeccionar qué documentos usó:
    # for i, d in enumerate(resp.get("context", []), 1):
    #     print(f"[{i}] {d.metadata.get('source', 'doc')}:{d.metadata.get('page', '')}")

print("Pregunta 1: ¿Qué es RAG?")
conversar("¿Qué es RAG?")

print("\nPregunta 2: ¿Y cómo se relaciona con LangChain? (nota: usa un pronombre)")
conversar("¿Y cómo se relaciona con LangChain?")

print("\nPregunta 3: ¿Dónde mencionaste los retrievers?")
conversar("¿Dónde mencionaste los retrievers?")


Pregunta 1: ¿Qué es RAG?
→ RAG (Retrieval-Augmented Generation) combina recuperación de contexto con generación. Permite responder con información actualizada sin reentrenar el modelo.

Pregunta 2: ¿Y cómo se relaciona con LangChain? (nota: usa un pronombre)
→ No aparece.

Pregunta 3: ¿Dónde mencionaste los retrievers?
→ Los retrievers se mencionan en el contexto como aquellos que devuelven documentos relevantes.


### Consultas la base de vectores

In [ ]:
query = "¿Qué es RAG?"
pairs = vectordb.similarity_search_with_score(query, k=4)
for doc, score in pairs:
    print(score, doc.metadata.get("source"))

0.8152427673339844 data/intro_rag.txt
1.3764668703079224 data/langchain_nociones.md


In [ ]:
query = "¿Qué es RAG?"
docs = retriever.invoke(query)  # equivalente a retriever.get_relevant_documents(query)

for i, d in enumerate(docs, 1):
    print(f"[{i}] {d.metadata.get('source', 'doc')}  pág: {d.metadata.get('page')}")
    print(d.page_content[:300], "...\n")

[1] data/intro_rag.txt  pág: None
RAG (Retrieval-Augmented Generation) combina recuperación de contexto con generación. Permite responder con información actualizada sin reentrenar el modelo. ...

[2] data/langchain_nociones.md  pág: None
# Nociones de LangChain
- LCEL permite encadenar componentes con el operador |.
- Los retrievers devuelven documentos relevantes.
- Los vector stores como Chroma y FAISS almacenan embeddings. ...




### Notas y buenas prácticas
- Si ves advertencias de *deprecations*, revisa las guías de migración de LangChain y actualiza imports.
- Para **persistir** historiales entre sesiones, puedes reemplazar `ChatMessageHistory` por alternativas como `FileChatMessageHistory` o una base de datos.
- **FAISS** es otra base vectorial popular (`faiss-cpu`). El código cambia poco: `from langchain_community.vectorstores import FAISS` y `FAISS.from_documents(...)`.
- Ajusta `chunk_size`/`chunk_overlap` según el tamaño de tus documentos y el contexto del modelo.



---

## Créditos y recursos
- Documentación de **ChatOpenAI** y `langchain-openai` (instalación y uso).
- `ConversationBufferMemory` (estado: deprecado, pero útil para aprender).
- RAG moderno con `create_history_aware_retriever` y `create_retrieval_chain`.
- `create_stuff_documents_chain` para combinar documentos en el prompt.

> Revisa las guías oficiales para ejemplos actualizados, cambios en APIs y mejores prácticas.


### Función de ayuda: `preguntar()`

In [ ]:

def preguntar(texto: str, session_id: str = "demo", mostrar_fuentes: bool = True):
    if 'rag_with_history' in globals():
        config = {"configurable": {"session_id": session_id}}
        resp = rag_with_history.invoke({"input": texto}, config=config)
    elif 'rag_chain' in globals():
        resp = rag_chain.invoke({"input": texto, "chat_history": []})
    else:
        raise RuntimeError("No encontré 'rag_with_history' ni 'rag_chain'. Ejecutá las celdas de la sección 3.4 y 3.5.")

    print(resp.get("answer") or resp)
    if mostrar_fuentes and isinstance(resp, dict) and "context" in resp:
        print("\nFuentes:")
        for i, d in enumerate(resp["context"], 1):
            src = d.metadata.get("source", "doc")
            page = d.metadata.get("page", "")
            print(f"[{i}] {src} {('pág ' + str(page)) if page not in (None, '') else ''}")
    return resp
